In [1]:
import numpy as np
import pandas as pd
import torch.nn as nn
import PIL
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True #imagenes ligeramente corruptas OK
from transformers import AutoImageProcessor
from transformers import AutoModelForImageClassification

In [2]:
data=pd.read_feather("data/stage1_partial.feather")

In [3]:
data["predicted"]=np.full(data.shape[0],"")

In [4]:
data["certainty"]=np.full(data.shape[0],"")

In [5]:
model_name="./models-legacy/swinv2-base-patch4-window16-256-retrained-f1-85/"

In [6]:
device="cuda"

In [7]:
model = AutoModelForImageClassification.from_pretrained(model_name).to(device)

In [8]:
image_processor = AutoImageProcessor.from_pretrained(model_name)

In [9]:
def load_image(path):
    return PIL.Image.open(path)

In [10]:
def process(image):
    return image_processor(images=image, return_tensors="pt").to(model.device)["pixel_values"]

In [11]:
def consume(pixels):
    logits=model(pixel_values=pixels).logits
    return nn.functional.softmax(logits, dim=-1).detach().cpu().numpy().flatten()

In [12]:
def classify(probabilities):
    idx=probabilities.argmax(-1).item()
    return model.config.id2label[idx]

In [13]:
from fastprogress import progress_bar as pb

In [ ]:
for idx,r in pb(data.iterrows(), data.shape[0]):
    try:
        probs=consume(process(load_image(r["path"])))
        cl=classify(probs)
        crt=probs.max()
    except Exception as E:
        print("ERROR:",E)
        print(r)
        cl="UNK"
        crt=0
    data.loc[idx,"predicted"]=cl
    data.loc[idx,"certainty"]=crt

ERROR: Unsupported number of image dimensions: 2
path                 images\2000\dic\p1215003.jpg
volcano_certainty                               0
day_night                                       0
has_fume                                        0
is_explosion                                    0
predicted                                        
certainty                                        
Name: 135, dtype: object
ERROR: Unsupported number of image dimensions: 2
path                 images\2000\dic\p1219008.jpg
volcano_certainty                               0
day_night                                       0
has_fume                                        0
is_explosion                                    0
predicted                                        
certainty                                        
Name: 156, dtype: object
ERROR: Unsupported number of image dimensions: 2
path                 images\2000\dic\p122400a.jpg
volcano_certainty                               0
day

In [ ]:
data